In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class Process:
    pid: str
    allocation: list
    maximum: list
    need: list = None
    finished: bool = False

# ---------- Metrics + Table ----------

def calculate_metrics(processes: List[Process], timeline: List[Tuple[str, int, int]]):
    # For Banker's Algorithm, we can show the allocation, max, need, and finished status
    df = pd.DataFrame([{
        "PID": p.pid,
        "Allocation": p.allocation,
        "Max": p.maximum,
        "Need": p.need,
        "Finished": p.finished
    } for p in processes])
    display(df)

# ---------- Gantt Chart ----------

def draw_gantt_chart(timeline: List[Tuple[str, int, int]], title: str):
    # Not meaningful for Banker's, but keep for compatibility
    print(f"Safe Sequence: {' -> '.join(pid for pid,_,_ in timeline)}")

# ---------- Banker's Algorithm ----------

def bankers_algorithm(processes: List[Process], available: list):
    n = len(processes)
    m = len(available)
    for p in processes:
        p.need = [p.maximum[i] - p.allocation[i] for i in range(m)]
        p.finished = False
    work = available.copy()
    timeline = []
    safe_sequence = []
    while len(safe_sequence) < n:
        allocated = False
        for p in processes:
            if not p.finished and all(p.need[i] <= work[i] for i in range(m)):
                timeline.append((p.pid, 0, 0))  # No real time, just sequence
                work = [work[i] + p.allocation[i] for i in range(m)]
                p.finished = True
                safe_sequence.append(p.pid)
                allocated = True
                break
        if not allocated:
            break
    print("\n=== Banker's Algorithm ===")
    if len(safe_sequence) == n:
        draw_gantt_chart([(pid,0,0) for pid in safe_sequence], "Banker's Safe Sequence")
    else:
        print("No safe sequence exists. System is in UNSAFE state.")
    calculate_metrics(processes, timeline)

# ---------- Example Run ----------

processes = [
    Process("P1", [0,1,0], [7,5,3]),
    Process("P2", [2,0,0], [3,2,2]),
    Process("P3", [3,0,2], [9,0,2]),
    Process("P4", [2,1,1], [2,2,2]),
    Process("P5", [0,0,2], [4,3,3]),
]
available = [3,3,2]

bankers_algorithm([Process(p.pid, p.allocation, p.maximum) for p in processes], available)


=== Banker's Algorithm ===
Safe Sequence: P2 -> P4 -> P1 -> P3 -> P5


,PID,Allocation,Max,Need,Finished
0,P1,"[0, 1, 0]","[7, 5, 3]","[7, 4, 3]",True
1,P2,"[2, 0, 0]","[3, 2, 2]","[1, 2, 2]",True
2,P3,"[3, 0, 2]","[9, 0, 2]","[6, 0, 0]",True
3,P4,"[2, 1, 1]","[2, 2, 2]","[0, 1, 1]",True
4,P5,"[0, 0, 2]","[4, 3, 3]","[4, 3, 1]",True
